## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
import numpy as np
from math import sqrt
from pyspark.ml.clustering import KMeans, KMeansModel
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets.samples_generator import make_blobs
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
#reading files to df
df1 = spark.read.format("csv")\
       .option("header", "true")\
       .option("inferSchema", "true")\
       .load("dbfs:/FileStore/tables/*.csv") 

In [4]:
df1.printSchema()

root
-- date: timestamp (nullable = true)
-- serial_number: string (nullable = true)
-- model: string (nullable = true)
-- capacity_bytes: long (nullable = true)
-- failure: integer (nullable = true)
-- smart_1_normalized: integer (nullable = true)
-- smart_1_raw: integer (nullable = true)
-- smart_2_normalized: integer (nullable = true)
-- smart_2_raw: integer (nullable = true)
-- smart_3_normalized: integer (nullable = true)
-- smart_3_raw: integer (nullable = true)
-- smart_4_normalized: integer (nullable = true)
-- smart_4_raw: integer (nullable = true)
-- smart_5_normalized: integer (nullable = true)
-- smart_5_raw: integer (nullable = true)
-- smart_7_normalized: integer (nullable = true)
-- smart_7_raw: long (nullable = true)
-- smart_8_normalized: integer (nullable = true)
-- smart_8_raw: integer (nullable = true)
-- smart_9_normalized: integer (nullable = true)
-- smart_9_raw: integer (nullable = true)
-- smart_10_normalized: integer (nullable = true)
-- smart_10_raw: integer (nullable = true)
-- smart_11_normalized: integer (nullable = true)
-- smart_11_raw: integer (nullable = true)
-- smart_12_normalized: integer (nullable = true)
-- smart_12_raw: integer (nullable = true)
-- smart_13_normalized: string (nullable = true)
-- smart_13_raw: string (nullable = true)
-- smart_15_normalized: string (nullable = true)
-- smart_15_raw: string (nullable = true)
-- smart_16_normalized: integer (nullable = true)
-- smart_16_raw: integer (nullable = true)
-- smart_17_normalized: integer (nullable = true)
-- smart_17_raw: integer (nullable = true)
-- smart_22_normalized: integer (nullable = true)
-- smart_22_raw: integer (nullable = true)
-- smart_23_normalized: integer (nullable = true)
-- smart_23_raw: integer (nullable = true)
-- smart_24_normalized: integer (nullable = true)
-- smart_24_raw: integer (nullable = true)
-- smart_168_normalized: integer (nullable = true)
-- smart_168_raw: integer (nullable = true)
-- smart_170_normalized: integer (nullable = true)
-- smart_170_raw: integer (nullable = true)
-- smart_173_normalized: integer (nullable = true)
-- smart_173_raw: long (nullable = true)
-- smart_174_normalized: integer (nullable = true)
-- smart_174_raw: integer (nullable = true)
-- smart_177_normalized: integer (nullable = true)
-- smart_177_raw: integer (nullable = true)
-- smart_179_normalized: string (nullable = true)
-- smart_179_raw: string (nullable = true)
-- smart_181_normalized: string (nullable = true)
-- smart_181_raw: string (nullable = true)
-- smart_182_normalized: string (nullable = true)
-- smart_182_raw: string (nullable = true)
-- smart_183_normalized: integer (nullable = true)
-- smart_183_raw: integer (nullable = true)
-- smart_184_normalized: integer (nullable = true)
-- smart_184_raw: integer (nullable = true)
-- smart_187_normalized: integer (nullable = true)
-- smart_187_raw: integer (nullable = true)
-- smart_188_normalized: integer (nullable = true)
-- smart_188_raw: long (nullable = true)
-- smart_189_normalized: integer (nullable = true)
-- smart_189_raw: integer (nullable = true)
-- smart_190_normalized: integer (nullable = true)
-- smart_190_raw: integer (nullable = true)
-- smart_191_normalized: integer (nullable = true)
-- smart_191_raw: integer (nullable = true)
-- smart_192_normalized: integer (nullable = true)
-- smart_192_raw: integer (nullable = true)
-- smart_193_normalized: integer (nullable = true)
-- smart_193_raw: integer (nullable = true)
-- smart_194_normalized: integer (nullable = true)
-- smart_194_raw: integer (nullable = true)
-- smart_195_normalized: integer (nullable = true)
-- smart_195_raw: integer (nullable = true)
-- smart_196_normalized: integer (nullable = true)
-- smart_196_raw: integer (nullable = true)
-- smart_197_normalized: integer (nullable = true)
-- smart_197_raw: integer (nullable = true)
-- smart_198_normalized: integer (nullable = true)
-- smart_198_raw: integer (nullable = true)
-- smart_199_normalized: integer (nullable = true)
-- smart_199_raw: in

In [5]:
print('total number of rows and columns in dataframe:')
(df1.count() , len(df1.columns))

total number of rows and columns in dataframe:
Out[6]: (9577046, 129)

In [6]:
#calculating aggregate of NA values
df7 = df1.select(['model','smart_1_normalized','failure'])
data_agg = df7.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df7.columns])
data_agg.show()

+-----+------------------+-------+
model|smart_1_normalized|failure|
+-----+------------------+-------+
 0| 1534| 0|
+-----+------------------+-------+

In [7]:
df4 = df1.select('serial_number','smart_1_normalized')

In [8]:
#replace null values by zero
df4 = df4.fillna(0)
df4.show()

+--------------+------------------+
 serial_number|smart_1_normalized|
+--------------+------------------+
 Z305B2QN| 117|
 ZJV0XJQ4| 80|
 ZJV0XJQ3| 83|
 ZJV0XJQ0| 81|
PL1331LAHG1S4H| 100|
 ZA16NQJR| 75|
 ZJV02XWG| 83|
 ZJV1CSVX| 83|
 ZJV02XWA| 78|
 ZA18CEBS| 77|
 Z305DEMG| 117|
 ZA130TTW| 81|
 ZJV1CSVV| 74|
 ZA18CEBF| 80|
 ZJV02XWV| 78|
PL2331LAG9TEEJ| 100|
PL2331LAH3WYAJ| 100|
PL1331LAHG53YH| 100|
 88Q0A0LGF97G| 100|
PL2331LAHDUVVJ| 100|
+--------------+------------------+
only showing top 20 rows

In [9]:
#using vector assembler to get the feature column later to be used for k-means
from pyspark.ml.feature import VectorAssembler,VectorIndexer
assembler = VectorAssembler(inputCols=['smart_1_normalized'], outputCol="smart_vec")
# fill the null values
df3_algo = assembler.transform(df4)

In [10]:
df3_algo.show()

+--------------+------------------+---------+
 serial_number|smart_1_normalized|smart_vec|
+--------------+------------------+---------+
 Z305B2QN| 117| [117.0]|
 ZJV0XJQ4| 80| [80.0]|
 ZJV0XJQ3| 83| [83.0]|
 ZJV0XJQ0| 81| [81.0]|
PL1331LAHG1S4H| 100| [100.0]|
 ZA16NQJR| 75| [75.0]|
 ZJV02XWG| 83| [83.0]|
 ZJV1CSVX| 83| [83.0]|
 ZJV02XWA| 78| [78.0]|
 ZA18CEBS| 77| [77.0]|
 Z305DEMG| 117| [117.0]|
 ZA130TTW| 81| [81.0]|
 ZJV1CSVV| 74| [74.0]|
 ZA18CEBF| 80| [80.0]|
 ZJV02XWV| 78| [78.0]|
PL2331LAG9TEEJ| 100| [100.0]|
PL2331LAH3WYAJ| 100| [100.0]|
PL1331LAHG53YH| 100| [100.0]|
 88Q0A0LGF97G| 100| [100.0]|
PL2331LAHDUVVJ| 100| [100.0]|
+--------------+------------------+---------+
only showing top 20 rows

In [11]:
#Reference: https://rsandstroem.github.io/sparkkmeans.html 
#This code was taking too long
#cost = np.zeros(10)
#for k in range(2,8):
  #  kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("smart_vec")
  #  model = kmeans.fit(df3_algo.sample(False,0.1, seed=25))
#    cost[k] = model.computeCost(df3_algo)

In [12]:
#here I have taken k = 5 please note that it was taking too long for the above process as I was doing it on databricks
#Reference: https://rsandstroem.github.io/sparkkmeans.html 
k = 5
kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("smart_vec")
model1 = kmeans.fit(df3_algo)
centers1 = model1.clusterCenters()

print("Cluster Centers: ")
for center in centers1:
    print(center)

Cluster Centers: 
[100.38532512]
[74.57268377]
[199.99904928]
[116.09234413]
[81.7848098]

In [13]:
#adding values of clusters to dataframe
k_transform = model1.transform(df3_algo)
k_transform.show(10)

+--------------+------------------+---------+----------+
 serial_number|smart_1_normalized|smart_vec|prediction|
+--------------+------------------+---------+----------+
 Z305B2QN| 117| [117.0]| 3|
 ZJV0XJQ4| 80| [80.0]| 4|
 ZJV0XJQ3| 83| [83.0]| 4|
 ZJV0XJQ0| 81| [81.0]| 4|
PL1331LAHG1S4H| 100| [100.0]| 0|
 ZA16NQJR| 75| [75.0]| 1|
 ZJV02XWG| 83| [83.0]| 4|
 ZJV1CSVX| 83| [83.0]| 4|
 ZJV02XWA| 78| [78.0]| 1|
 ZA18CEBS| 77| [77.0]| 1|
+--------------+------------------+---------+----------+
only showing top 10 rows

In [14]:
k_transform.select('prediction').describe().show()

+-------+------------------+
summary| prediction|
+-------+------------------+
 count| 9577046|
 mean| 2.349977226798326|
 stddev|1.6690788729139039|
 min| 0|
 max| 4|
+-------+------------------+

In [15]:
#taking approximate values of centers
c1 = 100
c2 = 74
c3 = 200
c4 = 116
c5 = 81
centers_01 = np.array([[c1,0], [c2,1], [c3,2],[c4,3],[c5,4]])
centers2 = sc.parallelize(centers_01)
centers3 = centers2.map(lambda x: [int(i) for i in x])
df3_centers = centers3.toDF(["center","prediction"])
df3_centers.show()

+------+----------+
center|prediction|
+------+----------+
 100| 0|
 74| 1|
 200| 2|
 116| 3|
 81| 4|
+------+----------+

In [16]:
#perform simple join, dataframe is sorted due to it so values appear similar
k_means1 = k_transform.join(df3_centers, 'prediction')
k_means1.show()

+----------+--------------+------------------+---------+------+
prediction| serial_number|smart_1_normalized|smart_vec|center|
+----------+--------------+------------------+---------+------+
 0|PL1331LAHG1S4H| 100| [100.0]| 100|
 0|PL2331LAG9TEEJ| 100| [100.0]| 100|
 0|PL2331LAH3WYAJ| 100| [100.0]| 100|
 0|PL1331LAHG53YH| 100| [100.0]| 100|
 0| 88Q0A0LGF97G| 100| [100.0]| 100|
 0|PL2331LAHDUVVJ| 100| [100.0]| 100|
 0|PL1331LAHD252H| 100| [100.0]| 100|
 0|PL1331LAHD1HTH| 100| [100.0]| 100|
 0| 8CGDP8AH| 100| [100.0]| 100|
 0| AAG6EYAH| 100| [100.0]| 100|
 0| AAG6E3AH| 100| [100.0]| 100|
 0|PL1331LAHD00PH| 100| [100.0]| 100|
 0| ZJV0XJQR| 100| [100.0]| 100|
 0|PL2331LAGPS2TJ| 100| [100.0]| 100|
 0| 78B0A052F97G| 100| [100.0]| 100|
 0|PL2331LAHDE6VJ| 100| [100.0]| 100|
 0| 96HCTBI0T| 100| [100.0]| 100|
 0|PL1331LAHBN1UH| 100| [100.0]| 100|
 0|PL1331LAHD2ADH| 100| [100.0]| 100|
 0|PL2331LAHDW6GJ| 100| [100.0]| 100|
+----------+--------------+------------------+---------+------+
only showing top 20 rows

In [17]:
k_means1.printSchema()

root
-- prediction: integer (nullable = false)
-- serial_number: string (nullable = true)
-- smart_1_normalized: integer (nullable = true)
-- smart_vec: vector (nullable = true)
-- center: long (nullable = true)

In [18]:
k_means1.select('prediction').distinct().show()

+----------+
prediction|
+----------+
 1|
 3|
 4|
 2|
 0|
+----------+

In [19]:
#setting max and min columns in the dataframe
k_means2 = k_means1.withColumn("Min",k_means1["center"]-100)
k_means2 = k_means2.withColumn("Max",k_means1["center"]+100)

In [20]:
#calculating euclidean distance of value from centroid
k_means3 = k_means2.withColumn("euclidean_dist",((k_means2["center"])**2-(k_means2['smart_1_normalized'])**2)**0.5)

In [21]:
k_means3.show()

+----------+--------------+------------------+---------+------+---+---+--------------+
prediction| serial_number|smart_1_normalized|smart_vec|center|Min|Max|euclidean_dist|
+----------+--------------+------------------+---------+------+---+---+--------------+
 0|PL1331LAHG1S4H| 100| [100.0]| 100| 0|200| 0.0|
 0|PL2331LAG9TEEJ| 100| [100.0]| 100| 0|200| 0.0|
 0|PL2331LAH3WYAJ| 100| [100.0]| 100| 0|200| 0.0|
 0|PL1331LAHG53YH| 100| [100.0]| 100| 0|200| 0.0|
 0| 88Q0A0LGF97G| 100| [100.0]| 100| 0|200| 0.0|
 0|PL2331LAHDUVVJ| 100| [100.0]| 100| 0|200| 0.0|
 0|PL1331LAHD252H| 100| [100.0]| 100| 0|200| 0.0|
 0|PL1331LAHD1HTH| 100| [100.0]| 100| 0|200| 0.0|
 0| 8CGDP8AH| 100| [100.0]| 100| 0|200| 0.0|
 0| AAG6EYAH| 100| [100.0]| 100| 0|200| 0.0|
 0| AAG6E3AH| 100| [100.0]| 100| 0|200| 0.0|
 0|PL1331LAHD00PH| 100| [100.0]| 100| 0|200| 0.0|
 0| ZJV0XJQR| 100| [100.0]| 100| 0|200| 0.0|
 0|PL2331LAGPS2TJ| 100| [100.0]| 100| 0|200| 0.0|
 0| 78B0A052F97G| 100| [100.0]| 100| 0|200| 0.0|
 0|PL2331LAHDE6VJ| 100| [100.0]| 100| 0|200| 0.0|
 0| 96HCTBI0T| 100| [100.0]| 100| 0|200| 0.0|
 0|PL1331LAHBN1UH| 100| [100.0]| 100| 0|200| 0.0|
 0|PL1331LAHD2ADH| 100| [100.0]| 100| 0|200| 0.0|
 0|PL2331LAHDW6GJ| 100| [100.0]| 100| 0|200| 0.0|
+----------+--------------+------------------+---------+------+---+---+--------------+
only showing top 20 rows

In [23]:
#taking columns to display a scatter plot 
xx = k_means3.select('prediction')
yy = k_means3.select('smart_1_normalized')
xx = xx.toPandas()
yy = yy.toPandas()
xx = xx.values
yy = yy.values

In [24]:
%matplotlib inline

plt.scatter(xx,yy)